# 0. 라이브러리 & 데이터

## 0.1 라이브러리

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False

import warnings
warnings.simplefilter("ignore")

## 0.2 데이터

In [3]:
case_fraud = pd.read_csv('../1_Data/case_fraud.csv')
case_fraud.head(1)

,index,판례일련번호,사건명,사건번호,선고일자,법원명,사건종류명,사건종류코드,판결유형,선고,판례상세링크,판시사항,판결요지,참조조문,참조판례,판례내용
0,20,238021,특정경제범죄가중처벌등에관한법률위반(사기)·사기,2023도12424,2023.11.16,대법원,형사,400102.0,판결,선고,/DRF/lawService.do?OC=yeorii&target=prec&ID=23...,"유사수신행위를 금지·처벌하는 유사수신행위의 규제에 관한 법률 제6조 제1항, 제3...","유사수신행위의 규제에 관한 법률(이하 ‘유사수신행위법’이라 한다) 제6조 제1항, ...","유사수신행위의 규제에 관한 법률 제3조, 제6조 제1항, 형법 제37조, 제347조...",내용없음,【피 고 인】 피고인【상 고 인】 피고인【변 호 인】 변호사 장용배【배상신청인】 배...


# 1. 판례내용 전처리

## 1.1 이유 이후로 자르기

In [4]:
case_fraud['판례내용_전처리'] = case_fraud.판례내용.str.split('유】|유]').str[1]

In [5]:
case_fraud.판례내용_전처리[:3]

0      상고이유를 판단한다.  1. 불가벌적 사후행위에 대한 판단「유사수신행위의 규제에...
1      상고이유를 판단한다.   1. 피고인 1, 피고인 2, 피고인 3, 피고인 4,...
2      직권으로 판단한다.   1. 의료법 위반의 점에 관한 공소사실의 요지   피고인...
Name: 판례내용_전처리, dtype: object

## 1.2 한자 처리

https://github.com/suminb/hanja

In [6]:
import hanja
from hanja import hangul

In [7]:
hanja.is_hanja('潘')

True

In [8]:
def hanja_to_hangul(sent):
    return hanja.translate(sent, 'substitution')

In [9]:
case_fraud['판례내용_전처리_한자'] = case_fraud.판례내용_전처리.apply(hanja_to_hangul)

In [10]:
print((case_fraud['판례내용_전처리_한자'] != case_fraud['판례내용_전처리']).sum())
case_fraud[case_fraud['판례내용_전처리_한자'] != case_fraud['판례내용_전처리']].판례내용_전처리

116


12        상고이유를 판단한다.  1. 사건의 개요와 쟁점  가. 이 사건 공소사실 중 도...
15        상고이유를 판단한다.  1. 공소사실의 요지  피고인은 2019. 1. 22. ...
34      1. 항소이유의 요지가. 사실오인, 법리오해1) 공모 사실 및 편취 고의의 부존재 ...
64      I. 항소이유의 개요1. 피고인들가. 사실오인, 법리오해        원심은 아래와...
88      1. ○○대 총장 표창장 위조(2019고합738호)  가. 공소사실    피고인은 ...
89      1. 이 법원의 심판범위   제2 원심은 피고인에 대한 공소사실 중 공소외 3, 공...
105       상고이유(상고이유서 제출기한이 지난 후에 제출된 피고인 2의 각 상고이유 보충서...
106       상고이유를 판단한다.  1. 배임의 점에 대하여  가. 배임죄는 타인의 사무를 ...
112       1. 이 사건 공소사실의 요지는, 피고인 1은 2009년 공소외 1을 만나기 전...
132       상고이유를 판단한다.   1. 석유 및 석유대체연료 사업법 위반 부분  가. 이...
146       상고이유(상고이유서 제출기간이 지난 다음에 제출된 서면들은 상고이유를 보충하는 ...
157       상고이유를 판단한다.  1. 가. 국민건강보험법은 국민의 질병·부상에 대한 예방...
166       상고이유(상고이유서 제출기간이 지난 다음 제출된 상고이유보충서는 이를 보충하는 ...
168     【범죄사실(주1)】범죄사실[범죄전력]  피고인은 2018. 1. 19. 서울고등법원...
191     Ⅰ. 항소이유 개요1. 피고인 1의 항소이유 개요가. 소송절차 관련 주장1) 공소제...
192     1. 이 사건 공소사실피고인 1은 가수 겸 화가로서 1973년 첫 개인전을 개최한 ...
197       상고이유를 판단한다.  1. 이 사건 쟁점과 관련된 공소사실 요지와 원심판단은 ...
220     【범죄사실(

In [11]:
case_fraud.판례내용_전처리 = case_fraud.판례내용_전처리.apply(hanja_to_hangul)

## 1.3 한글 외의 특수 문자 제거

In [12]:
import re

def remove_special_char(sent):
    return re.sub('[^가-힣 ]', '', sent)

In [13]:
case_fraud.판례내용_전처리 = case_fraud.판례내용_전처리.apply(remove_special_char)
case_fraud.판례내용_전처리[:3]

0      상고이유를 판단한다   불가벌적 사후행위에 대한 판단유사수신행위의 규제에 관한 ...
1      상고이유를 판단한다    피고인  피고인  피고인  피고인  피고인 에 대한 의...
2      직권으로 판단한다    의료법 위반의 점에 관한 공소사실의 요지   피고인은 의...
Name: 판례내용_전처리, dtype: object

In [14]:
case_fraud['판례내용_전처리'][0]

'  상고이유를 판단한다   불가벌적 사후행위에 대한 판단유사수신행위의 규제에 관한 법률이하 유사수신행위법이라 한다 제조 제항 제조를 위반한 행위는 그 자체가 사기행위에 해당한다거나 사기행위를 반드시 포함한다고 할 수 없고 유사수신행위법 위반죄가 형법 제조 제항의 사기죄와 구성요건을 달리하는 별개의 범죄로서 서로 보호법익이 다른 이상 유사수신행위를 한 자가 출자자에게 별도의 기망행위를 하여 유사수신행위로 조달받은 자금의 전부 또는 일부를 다시 투자받는 행위는 유사수신행위법 위반죄와 다른 새로운 보호법익을 침해하는 것으로서 유사수신행위법 위반죄의 불가벌적 사후행위가 되는 것이 아니라 별죄인 사기죄를 구성한다  원심은 판시와 같은 이유로 피고인의   경 피해자 공소외인에 대한 판시 사기의 점에 관한 공소사실에 대하여 유사수신행위법 제조에서 금지하는 유사수신행위에는 기망행위가 포함되어 있지 않고 유사수신행위법 위반죄와 사기죄는 구성요건을 달리하는 별개의 범죄로서 서로 행위의 태양이나 보호법익을 달리한다는 등의 이유로 기존 범죄인 유사수신행위법 위반죄의 가벌적 평가범위 내에 흡수되는 불가벌적 사후행위에 해당하지 않는다고 판단하였다  원심판결 이유를 위 법리 및 기록에 비추어 살펴보면 이 부분 원심판단에 일부 적절하지 않은 점이 있으나 피고인의   경 피해자 공소외인에 대한 판시 사기죄가 유사수신행위법 위반죄의 불가벌적 사후행위에 해당하지 않는다고 본 원심의 판단에 불가벌적 사후행위에 관한 법리를 오해함으로써 판결에 영향을 미친 잘못이 없다   나머지 상고이유에 대한 판단  이 부분 상고이유는 형사소송법 제조 제호에서 정한 형보다 가벼운 형이 선고된 이 사건에서 사실오인법리오해를 내세우며 실질적으로 원심의 증거 선택 및 증명력에 관한 판단 또는 이에 기초한 사실인정을 탓하는 것이거나 원심이 인정한 사실과 다른 사실관계를 전제로 법리오해를 지적하는 취지의 주장 또는 형이 너무 무거워 부당하다는 취지의 주장에 해당하여 모두 적법한 상고이유로 볼 수 없다   결론  그러므로

## [Skip] 띄어쓰기 처리

In [15]:
# from pykospacing import Spacing

# spacing = Spacing() 

In [16]:
# spacing(case_fraud['판례내용_전처리'][0])

## 1.4 토크나이저

### 1.4.1 토크나이저 비교

#### [Skip] SoyNLP

In [17]:
# import urllib.request
# from soynlp import DoublespaceLineCorpus
# from soynlp.word import WordExtractor

In [18]:
# # 훈련 데이터를 다수의 문서로 분리
# # urllib.request.urlretrieve("https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt", filename="2016-10-20.txt")
# corpus = DoublespaceLineCorpus("2016-10-20.txt")
# len(corpus)

In [19]:
# # 학습하기
# word_extractor = WordExtractor()
# word_extractor.train(corpus)
# word_score_table = word_extractor.extract()
# scores = {word:score.cohesion_forward for word, score in word_score_table.items()}

In [20]:
# from soynlp.tokenizer import MaxScoreTokenizer

# maxscore_tokenizer = MaxScoreTokenizer(scores=scores)

#### KoNLPy

In [21]:
import konlpy
konlpy.jvm.init_jvm(jvmpath=None, max_heap_size=2048)

[0.038s][warning][os,thread] Attempt to protect stack guard pages failed (0x000000016b448000-0x000000016b454000).
[0.038s][warning][os,thread] Attempt to deallocate stack guard pages failed.


In [22]:
from konlpy.tag import *

hannanum = Hannanum()
kkma = Kkma()
komoran = Komoran()
okt = Okt()

#### 비교

In [23]:
sent = case_fraud.판례내용_전처리[0][:300]

print('Input:', sent, end='\n\n')

#print('Hannanum:', hannanum.morphs(sent), end='\n\n')
print(kkma.morphs(sent), end='\n\n')
print(komoran.morphs(sent), end='\n\n')
# print(okt.morphs(sent), end='\n\n')
#print('SoyNLP:', maxscore_tokenizer.tokenize(sent))

Input:   상고이유를 판단한다   불가벌적 사후행위에 대한 판단유사수신행위의 규제에 관한 법률이하 유사수신행위법이라 한다 제조 제항 제조를 위반한 행위는 그 자체가 사기행위에 해당한다거나 사기행위를 반드시 포함한다고 할 수 없고 유사수신행위법 위반죄가 형법 제조 제항의 사기죄와 구성요건을 달리하는 별개의 범죄로서 서로 보호법익이 다른 이상 유사수신행위를 한 자가 출자자에게 별도의 기망행위를 하여 유사수신행위로 조달받은 자금의 전부 또는 일부를 다시 투자받는 행위는 유사수신행위법 위반죄와 다른 새로운 보호법익을 침해하는 것으로서 유사수신행위

['상고', '이유', '를', '판단', '하', 'ㄴ다', '불가', '벌', 'ㄹ', '적', '사후', '행위', '에', '대하', 'ㄴ', '판단', '유사', '수신', '행위', '의', '규제', '에', '관하', 'ㄴ', '법률', '이하', '유사', '수신', '행위법', '이', '라', '하', 'ㄴ다', '제조', '제항', '제조', '를', '위반', '하', 'ㄴ', '행위', '는', '그', '자체', '가', '사기', '행위', '에', '해당', '하', 'ㄴ다', '거나', '사기', '행위', '를', '반드시', '포함', '하', 'ㄴ다고', '하', 'ㄹ', '수', '없', '고', '유사', '수신', '행위법', '위반죄', '가', '형법', '제조', '제항', '의', '사기죄', '와', '구성', '요건', '을', '달리하', '는', '별개', '의', '범죄', '로서', '서로', '보호', '법익', '이', '다른', '이상', '유사', '수신', '행위', '를', '하', 'ㄴ', '자가', '출자자', '에게', '별도', '의', '기망', '행위', '를', '하여', '유사', '수신', '행위', '로', '조달', '받', '은', '자금', '의', '전부', '또는', '일부', '를', '다시', '투자', '받', '는

In [24]:
sent = case_fraud.판례내용_전처리[34][:300]

print('Input:', sent, end='\n\n')

#print('Hannanum:', hannanum.morphs(sent), end='\n\n')
print(kkma.morphs(sent), end='\n\n')
print(komoran.morphs(sent), end='\n\n')
# print(okt.morphs(sent), end='\n\n')
#print('SoyNLP:', maxscore_tokenizer.tokenize(sent))

Input:  항소이유의 요지가 사실오인 법리오해 공모 사실 및 편취 고의의 부존재     가 피고인들은 환전업의 일환으로 현금을 전달받아 다시 이를 제자에게 전달하였다 피고인들은 공소사실 기재와 같이 보이스피싱 사기 범행을 공모한 사실이 없을 뿐만 아니라 전달받은 돈이 보이스피싱 사기 범행으로 취득한 것임을 인식하지 못하였으므로 편취의 고의를 가지고 위 범행에 가담하였다고 할 수 없다     나 피고인들에 대한 보이스피싱 사기 범행에 대한 공소사실을 유죄로 인정한 원심에는 사실을 오인하거나 공모공동정범 및 편취의 고의에 대한 법리를 오해한 위

['항소', '이유', '의', '요지', '가', '사실', '오인', '법리', '오해', '공모', '사실', '및', '편취', '고의', '의', '부', '존재', '가', '아', '피고인', '들', '은', '환전업', '의', '일환', '으로', '현금', '을', '전달', '받', '아', '다시', '이르', 'ㄹ', '제자', '에게', '전달', '하', '였', '다', '피고인', '들', '은', '공소', '사실', '기재', '와', '같이', '보이스', '피', '싱', '사기', '범행', '을', '공모', '하', 'ㄴ', '사실', '이', '없', '을', '뿐', '만', '아니', '라', '전달', '받', '은', '돈', '이', '보이스', '피', '싱', '사기', '범행', '으로', '취득', '하', 'ㄴ', '것', '이', 'ㅁ', '을', '인식', '하', '지', '못하', '였', '으므로', '편취', '의', '고의', '를', '가지', '고', '위', '범행', '에', '가담', '하', '였', '다고', '하', 'ㄹ', '수', '없', '다', '나', '아', '피고인', '들', '에', '대하', 'ㄴ', '보이스', '피', '싱', '사기', '범행', '에', '대하', 'ㄴ', '공소', '사실', '을', 

In [25]:
sent = case_fraud.판례내용_전처리[0][:300]

print('Input:', sent, end='\n\n')

#print('Hannanum:', hannanum.morphs(sent), end='\n\n')
print(kkma.nouns(sent), end='\n\n')
print(komoran.nouns(sent), end='\n\n')
# print(okt.morphs(sent), end='\n\n')
#print('SoyNLP:', maxscore_tokenizer.tokenize(sent))

Input:   상고이유를 판단한다   불가벌적 사후행위에 대한 판단유사수신행위의 규제에 관한 법률이하 유사수신행위법이라 한다 제조 제항 제조를 위반한 행위는 그 자체가 사기행위에 해당한다거나 사기행위를 반드시 포함한다고 할 수 없고 유사수신행위법 위반죄가 형법 제조 제항의 사기죄와 구성요건을 달리하는 별개의 범죄로서 서로 보호법익이 다른 이상 유사수신행위를 한 자가 출자자에게 별도의 기망행위를 하여 유사수신행위로 조달받은 자금의 전부 또는 일부를 다시 투자받는 행위는 유사수신행위법 위반죄와 다른 새로운 보호법익을 침해하는 것으로서 유사수신행위

['상고', '상고이유', '이유', '판단', '불가', '적', '사후', '사후행위', '행위', '판단유사수신행위', '유사', '수신', '규제', '법률', '법률이하', '이하', '유사수신행위법', '행위법', '제조', '제항', '위반', '자체', '사기', '사기행위', '해당', '포함', '수', '위반죄', '형법', '사기죄', '구성', '구성요건', '요건', '별개', '범죄', '서로', '보호', '보호법익', '법익', '이상', '유사수신행위', '자가', '출자자', '별도', '기망', '기망행위', '조달', '자금', '전부', '일부', '투자', '새', '침해']

['상고', '이유', '판단', '불가벌적 사후행위', '판단', '유사', '수신', '행위', '규제', '법률', '이하', '유사', '수신', '행위', '법', '제조', '항', '제조', '위반', '행위', '자체', '사기', '행위', '해당', '사기', '행위', '포함', '수', '유사', '수신', '행위', '법', '위반', '죄', '형법', '제조', '항의', '사기죄', '구성요건', '별개', '범죄', '보호법익', '이상', '유사', '수신', '행위', '자', '출자', '자', '별도', '기망행위', '유사', '수신', '행위', '조달', '자금

In [25]:
sent = case_fraud.판례내용_전처리[34][:300]

print(sent, end='\n\n')
#print(hannanum.nouns(sent), end='\n')
print(kkma.nouns(sent), end='\n\n')
print(komoran.nouns(sent), end='\n\n')
#print(okt.nouns(sent), end='\n')

 항소이유의 요지가 사실오인 법리오해 공모 사실 및 편취 고의의 부존재     가 피고인들은 환전업의 일환으로 현금을 전달받아 다시 이를 제자에게 전달하였다 피고인들은 공소사실 기재와 같이 보이스피싱 사기 범행을 공모한 사실이 없을 뿐만 아니라 전달받은 돈이 보이스피싱 사기 범행으로 취득한 것임을 인식하지 못하였으므로 편취의 고의를 가지고 위 범행에 가담하였다고 할 수 없다     나 피고인들에 대한 보이스피싱 사기 범행에 대한 공소사실을 유죄로 인정한 원심에는 사실을 오인하거나 공모공동정범 및 편취의 고의에 대한 법리를 오해한 위

['항소', '항소이유', '이유', '요지', '사실', '사실오인', '오인', '법리', '법리오해', '오해', '공모', '편취', '고의', '존재', '피고인', '환전업', '일환', '현금', '전달', '제자', '공소', '공소사실', '기재', '보이스', '보이스피싱', '피', '싱', '사기', '범행', '뿐', '돈', '취득', '인식', '위', '가담', '수', '유죄', '인정', '원심', '공모공동정범', '공동', '정범']

['항소', '이유', '요지', '사실', '오인', '법리', '오해', '공모', '사실', '고의', '존재', '피고인', '환전', '업', '일환', '현금', '전달', '제자', '전달', '피고인', '공소사실', '기재', '보이스', '피싱', '사기', '범행', '공모', '사실', '뿐', '전달', '돈', '보이스', '피싱', '사기', '범행', '취득', '것', '인식', '고의', '위', '범행', '가담', '수', '피고인', '보이스', '피싱', '사기', '범행', '공소사실', '유죄', '인정', '원심', '사실', '오인하', '공모', '공동정범', '고의', '법리', '오해', '위']



### 1.4.2 토큰화 전 문장 내 단어 개수 체크

In [26]:
case_fraud['판례내용_전처리_글자수'] = case_fraud['판례내용_전처리'].apply(lambda x: len(x))
case_fraud.판례내용_전처리_글자수.value_counts().sort_index() # 45자부터 467481자까지

45        1
154       1
156       1
159       1
203       1
227       1
242       1
271       1
277       1
293       1
307       2
318       1
319       1
320       2
324       1
325       1
329       1
332       1
339       2
345       1
346       1
347       1
349       1
357       1
363       1
369       1
372       1
375       1
380       1
397       1
399       2
401       1
405       2
406       1
409       1
413       1
417       1
420       1
421       1
422       1
423       1
426       1
427       1
430       1
432       1
434       1
438       1
447       1
453       1
455       1
460       1
464       1
468       2
471       2
474       1
478       1
482       1
483       1
485       1
486       1
487       2
490       1
491       1
493       2
498       1
503       1
506       1
508       1
511       1
513       1
516       1
518       1
519       1
521       1
523       1
524       1
527       2
531       1
537       1
540       1
542       1
545       1
546       1
548 

In [27]:
print((case_fraud['판례내용_전처리_글자수'] > 90000).sum()) # 9만 자 넘는 판례 15개 있음
case_fraud = case_fraud[case_fraud['판례내용_전처리_글자수'] < 90000] # 9만 자 넘는 판례 15개 제거
print((case_fraud['판례내용_전처리_글자수'] > 90000).sum())

15
0


### 1.4.3 토큰화 

In [29]:
tqdm.pandas()
case_fraud['판례내용_토큰화'] = case_fraud['판례내용_전처리'].progress_apply(komoran.morphs)

100%|██████████| 2253/2253 [11:02<00:00,  3.40it/s] 


In [31]:
tqdm.pandas()
case_fraud['판례내용_토큰화_꼬꼬마'] = case_fraud['판례내용_전처리'].progress_apply(kkma.morphs)

100%|██████████| 2253/2253 [34:55<00:00,  1.08it/s]  


In [39]:
case_fraud.rename(columns = {'판례내용_토큰화' : '판례내용_토큰화_코모란'}, inplace = True)
case_fraud.drop('판례내용_전처리_한자', axis=1)
case_fraud.head(1)

,index,판례일련번호,사건명,사건번호,선고일자,법원명,사건종류명,사건종류코드,판결유형,선고,판례상세링크,판시사항,판결요지,참조조문,참조판례,판례내용,판례내용_전처리,판례내용_전처리_한자,판례내용_전처리_글자수,판례내용_토큰화_코모란,판례내용_토큰화_꼬꼬마
0,20,238021,특정경제범죄가중처벌등에관한법률위반(사기)·사기,2023도12424,2023.11.16,대법원,형사,400102.0,판결,선고,/DRF/lawService.do?OC=yeorii&target=prec&ID=23...,"유사수신행위를 금지·처벌하는 유사수신행위의 규제에 관한 법률 제6조 제1항, 제3...","유사수신행위의 규제에 관한 법률(이하 ‘유사수신행위법’이라 한다) 제6조 제1항, ...","유사수신행위의 규제에 관한 법률 제3조, 제6조 제1항, 형법 제37조, 제347조...",내용없음,【피 고 인】 피고인【상 고 인】 피고인【변 호 인】 변호사 장용배【배상신청인】 배...,상고이유를 판단한다 불가벌적 사후행위에 대한 판단유사수신행위의 규제에 관한 ...,상고이유를 판단한다. 1. 불가벌적 사후행위에 대한 판단「유사수신행위의 규제에...,1066,"[상고, 이유, 를, 판단, 하, ㄴ다, 불가벌적 사후행위, 에, 대하, ㄴ, 판단...","[상고, 이유, 를, 판단, 하, ㄴ다, 불가, 벌, ㄹ, 적, 사후, 행위, 에,..."


In [40]:
case_fraud.to_csv('../1_Data/case_fraud_prep.csv', index=False)